## String Matching -- KMP (Knuth–Morris–Pratt algorithm)


Given two strings $A$ and $B$, find the first occurrence (if any) of $B$ in $A$. In other words, find the smallerst $k$, such that for all $i$, $1 \le i \le m$, we have $a_{k+i} = b_i$.

A straightforward string matching algorithm is $O(mn)$.

Any better algorithm? Hint: we may want to reduce the number of comparison of two characters that have been already compared before.

In [1]:
A = 'xyxxyxyxyyxyxyxyyxyxyxx'
B = 'xyxyyxyxyxx'

n = len(A)
m = len(B)

# O(mn) solution!
for i in range(n-m+1):
    stillTrue = True
    for j in range(m):
        if A[i+j] != B[j]:
            stillTrue = False
            break
    if stillTrue:
        print(i, B)

12 xyxyyxyxyxx


In [2]:
# just for printing the following table to explain
# not an algorithm.
for i in range(n-m+1):
    if i==0:
        print(" A", A)
        print(" B", B)
        print("---------------")
    print("{0:2d}".format(i+1), "", end="")
    for s in range(i):
        print(" ", end="")
    for j in range(m):
        if A[i+j] == B[j]:
            print (B[j], end="")
        else:
            print (B[j], end="")
            break

    print()

 A xyxxyxyxyyxyxyxyyxyxyxx
 B xyxyyxyxyxx
---------------
 1 xyxy
 2  x
 3   xy
 4    xyxyy
 5     x
 6      xyxyyxyxyxx
 7       x
 8        xyx
 9         x
10          x
11           xyxyy
12            x
13             xyxyyxyxyxx


Hint: we can analysis the structure of B to reduce the number of comparison.
Take a look at line 6 of the above example.

We know $B(10) = b_1 b_2 \dots b_{10}$ is exactly the same as $A[6 \dots 15]$. We want to determine exactly how many steps $B$ can be shifted to the right until there is some hope of another match. We determine this number by looking for a maximum *suffix* of $B(10)$ that is equal to a *prefix* of B. In this case, that suffix is of length 3 (xyx).

In [3]:
# just for printing the following table to explain
# not an algorithm.

# In the following example, 
# B(10) is shifted, one step at a time,
# and is compared to itself,
# until a prefix matches a suffix.
# We ignore b11, since it is mismatched.

B10 = B[:-1]

for i in range(len(B10)):
    if i==0:
        print("B10", B10)
        print("---------------")
    print(" {0:2d}".format(i+1), "", end="")
    for s in range(i):
        print(" ", end="")
    for j in range(len(B10)):
        if i+j > len(B10)-1:
            break
        if B10[i+j] == B10[j]:
            print (B10[j], end="")
        else:
            print (B10[j], end="")
            break

    print()
    

# For B(10),
# we find that there are some prefix == suffix
#
# 1 xyxyyxyxyx == xyxyyxyxyx
# 2 xyxyyxyxy != yxyyxyxyx
# 3 xyxyyxyx != xyyxyxyx
# 4 xyxyyxy != yyxyxyx
# 5 xyxyyx != yxyxyx
# 6 xyxyy != xyxyx
# 7 xyxy != yxyx
# 8 xyx == xyx
# 9 xy != yx
#10 x == x

# In this case, in the 2nd cell, at line 6,
# when b11 != a16,
# we can skip the comparisons on line 7 to 12 and half those on line 13.

B10 xyxyyxyxyx
---------------
  1 xyxyyxyxyx
  2  x
  3   xyx
  4    x
  5     x
  6      xyxyy
  7       x
  8        xyx
  9         x
 10          x


We want to devise a way to handle mismatches when they occur withour backtracking.
When a mismatch is encountered, we consult a table to find how far in B we backtrack.

For each $b_i$, we want to find the largest suffix of $B(i-1)$ that is equal to a prefix of $B(i-1)$. If the length of this suffix is $j$, then the mismatched character in $A$ can be matched againist $b_{j+1}$ directly. Because we already know that the most recent $j$ characters in $A$ match the begining of $B$. The table is called *next()*.

$next(i)$ = the max $j$ ($0 \lt j \lt i-1$) such that
$b_{i-j} b_{i-j+1} \dots b_{i-1} = B(j)$, and 0 if no such $j$ exists. 

For convenience, we define $next(1) = -1$.

In [4]:
# Lets take a look at the next table fisrt, then discuss how to compute the table.

_next = [-1, 0, 0, 1, 2, 0, 1, 2, 3, 4, 3]

for i in range(len(B)):
    print("{0:2d}".format(i+1), end="")
print()
for i in range(len(B)):
    print("{0:>2}".format(B[i]), end="")
print()
for i in range(len(_next)):
    print("{0:>2}".format(_next[i]), end="")

 1 2 3 4 5 6 7 8 91011
 x y x y y x y x y x x
-1 0 0 1 2 0 1 2 3 4 3

In [5]:
'''
A note for you!

If B[j] is mismatched, we look at B[1]~B[j-1],
we find the max prefix and suffix in B[1]~B[j-1].
It specifiies the number of window to slide for such mismatch of B[j].
We called it next[j].
We define next[1] = -1 to distinguish the base case.

In this case, next[10] = 4. (xyxy)
'''

'\nA note for you!\n\nIf B[j] is mismatched, we look at B[1]~B[j-1],\nwe find the max prefix and suffix in B[1]~B[j-1].\nIt specifiies the number of window to slide for such mismatch of B[j].\nWe called it next[j].\nWe define next[1] = -1 to distinguish the base case.\n\nIn this case, next[10] = 4. (xyxy)\n'

In [6]:
# Note, in python string starts from 0.

def string_match(A, B):
    global _next # assume we have it.
    
    n = len(A)
    m = len(B)
    
    i = 0 # start from A[i]
    j = 0 # start from B[j]

    while i < n: # run through A[0] to A[n-1]
        # if matched, continue next character
        if B[j] == A[i]:
            print(i, j, 'matched')
            j += 1
            i += 1
        else: # mismatch
            print(i, j, 'mismatched')
            
            if _next[j] == -1: # B[0] mismatch, keep using B[0] and new A[i+1]
                i += 1
            elif _next[j] == 0: # no way to skip, keep using A[i] and new B[0]
                j = 0
            else: # we have prefix/suffix of length _next[j], j can skip to _next[j] for next round
                j = _next[j] 
            print(i, j, 'next round')

        if j == m: # if B[j] goes to B[m], all chars in B are matched
            print(i, j)
            return  (i - m) # return index of A[i-m], which is the 1st index such that B is a substr of A starting at A[i-m]


In [7]:
for i in range(len(A)):
    print("{0:2d}".format(i), end="")
print()
for i in range(len(A)):
    print("{0:>2}".format(A[i]), end="")
print()
for i in range(len(B)):
    print("{0:>2}".format(B[i]), end="")
print()

print(string_match(A, B))

 0 1 2 3 4 5 6 7 8 910111213141516171819202122
 x y x x y x y x y y x y x y x y y x y x y x x
 x y x y y x y x y x x
0 0 matched
1 1 matched
2 2 matched
3 3 mismatched
3 1 next round
3 1 mismatched
3 0 next round
3 0 matched
4 1 matched
5 2 matched
6 3 matched
7 4 mismatched
7 2 next round
7 2 matched
8 3 matched
9 4 matched
10 5 matched
11 6 matched
12 7 matched
13 8 matched
14 9 matched
15 10 mismatched
15 3 next round
15 3 matched
16 4 matched
17 5 matched
18 6 matched
19 7 matched
20 8 matched
21 9 matched
22 10 matched
23 11
12


In [8]:
for i in range(len(B)):
    print("{0:2d}".format(i), end="")
print()
for i in range(len(B)):
    print("{0:>2}".format(B[i]), end="")
print()
for i in range(len(_next)):
    print("{0:>2}".format(_next[i]), end="")

 0 1 2 3 4 5 6 7 8 910
 x y x y y x y x y x x
-1 0 0 1 2 0 1 2 3 4 3

In [9]:
def compute_next(B):
    next_ = [0] * len(B)
    next_[0] = -1
    next_[1] = 0
    
    for i in range(2, len(B)):
        j = next_[i-1]
        while B[i-1] != B[j] and j >= 0:
            j = next_[j]
        
        if j == next_[i-1]: # no loop, best case
            next_[i] = next_[i-1] + 1
        elif j == -1: # loop to -1
            next_[i] = 0
        elif j == 0: # loop to somewhere 0
            next_[i] = 0
        else: # loop to somewhere
            next_[i] = j + 1
    
    return next_

compute_next(B)

[-1, 0, 0, 1, 2, 0, 1, 2, 3, 4, 3]